
# AES (Advanced Encrypton Standard) #

[Leírás](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.197.pdf)

DES (Feistel network alapú) leváltására

Spec. Rijndael network
substitution-permutation network

Kulcs méretek: 128, 192, 256 bits (van egyéb Rijndael network is, de ezek kerültek be a szabványba)

Blokkok:

$\left(\begin{array}{rrrr}
k_0 & k_4 & k_8    & k_{12} \\
k_1 & k_5 & k_9    & k_{13} \\
k_2 & k_6 & k_{10} & k_{14} \\
k_3 & k_7 & k_{11} & k_{15}
\end{array}\right)\quad k_i\in \mathbb{F}_{2^8}$

Kulcsok méretének megfelelően 10, 12, 14 lépésben kapjuk meg a titkosított blokkot

**Rijndael véges test** (RF):  $\mathbb{F}_2[x]/_{x^8+x^4+x^3+x+1}$

**Rotációk gyűrűje** (FR): $\mathbb{F}_2[x]/_{x^8+1}$

**SubBytes** (SBox): $c \mapsto M*tobin(c^{-1}) + V$, ahol $M = \left(\begin{array}{rrrrrrrr}
1 & 0 & 0 & 0 & 1 & 1 & 1 & 1 \\
1 & 1 & 0 & 0 & 0 & 1 & 1 & 1 \\
1 & 1 & 1 & 0 & 0 & 0 & 1 & 1 \\
1 & 1 & 1 & 1 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 & 1
\end{array}\right)$ és $V = \left(\begin{array}{rrrrrrrr}
0 & 1 & 1 & 0 & 0 & 0 & 1 & 1
\end{array}\right)$

![](sbox.png)

**AddRoundKey**:

![](addroundkey.png)

**ShiftRows**:

![](shiftrows.png)

**MixColumns**:

![](mixcolumns.png)

## AES ##

0. AddRoundKey
2. (r-1)x SubBytes, ShiftRows, MixClumns, AddRoundKey
3. SubBytes, ShiftRows, AddroundKey

In [ ]:
# From previous class
Z2x.<x> = PolynomialRing(GF(2))
RF.<y> = Z2x.quotient_ring(x^8+x^4+x^3+x+1) # Rijndael test
FR.<z> = Z2x.quotient_ring(x^8+1) # Ring for rotations

# SubBytes
m = z^4+z^3+z^2+z+1
v = z^6+z^5+z+1
def SBox(c):
    return RF((FR((1/c).list())*m + v).list()) if c != 0 else RF(v.list())
def InvSBox(c):
    return 1/RF(((FR(c.list())-v)/m).list())

def make_round_keys(key):
    rks = key
    N = len(key) # number of input words
    for i in range(N,4*(N+7)):
        nk = rks[i-N]
        pk = rks[i-1]
        if i % N == 0:
            nk = [nk[j] + SBox(pk[(j+1) % 4]) + y^int(i/N) for j in range(4)]
        elif N > 6 and i % N == 4:
            nk = [nk[j] + SBox(pk[j]) for j in range(4)]
        else:
            nk = [nk[j] + pk[j] for j in range(4)]
        rks.append(nk)
    return rks

# Create a random key
Nr = 6 # 4 -> 128 bit, 6-> 192bit, 8->256 bit
key = [[RF.random_element() for i in range(4)] for j in range(Nr)]
#Create round keys
round_keys = make_round_keys(key)

#ShiftRows
def shiftrows(d):
    return [[d[i][(j+i) % 4] for j in range(4)] for i in range(4)]


## Mix Columns ##

Minden oszlopot egy polinomnak tekintünk és beszorzunk egy konstans polinommal, ami megfelel a következő mátrixal való szorzásnak.

In [ ]:
print('Matrix for multiplication:\n',table([[f"{[3,1,1,2][(i+3-j)&3]:#0{4}x}" for i in range(4)] for j in range(4)]), sep='')


MCRR.<w> = PolynomialRing(RF)
MCR.<w> = MCRR.quotient_ring(w^4+1)
a = (y+1)*w^3+w^2+w+y;
print('a: ', end=''); pretty_print(a)
print('1/a: ', end=''); pretty_print(1/a)

def MixColumns(d, a):
    r = [[0 for j in range(4)] for i in range(4)];
    for i in range(4):
        cp = sum([d[i][j]*w^j for j in range(4)]) # create a polynomial in MCR
        cp = cp * a # mulitply with a
        cpl = cp.list() # write back to the column
        for j in range(4):
            r[i][j] = cpl[j]
    return r

ri = [[RF.random_element() for i in range(4)] for j in range(4)]
mixed = MixColumns(ri, a)
remixed = MixColumns(mixed, 1/a)
print('Input:', end=''); pretty_print(Matrix(ri))
print('After mixing columns:', end=''); pretty_print(Matrix(mixed))
print('After mixing columns using inverse:', end=''); pretty_print(Matrix(remixed))

## AddRoundKeys ##

In [ ]:
def AddRoundKeys(block, round_keys, round_number):
    return [[block[j][i]+round_keys[4*round_number+i][j] for i in range(4)] for j in range(4)]

print('Input:', end=''); pretty_print(Matrix(ri))
print('After keys added:', end=''); pretty_print(Matrix(AddRoundKeys(ri, round_keys, 2)))

### Feladatok ###

9. Írjuk meg a MixColumns függvényt és inverzét bináris adatokra. (Tekinthető 2db feladatnak.) 1-1p
8. Állítsuk össze az AES encrypt függvényt a fentiek alapján. - 3p
9. Állítsuk össze az AES decrypt függvényt a fentiek alapján. - 3p
9. Írjuk meg az AES encrypt/decrypt függvényeket bináris formában. - 3-3p

In [ ]:
print("9. feladat")

def MixColumnsBin(d, a):
    r = [[0 for j in range(4)] for i in range(4)]
    for i in range(4):

        for j in range(4):
            r[i][j] = cpl[j]


Z2x.<x> = PolynomialRing(GF(2))
RF.<y> = Z2x.quotient_ring(x^8+x^4+x^3+x+1) # Rijndael test
FR.<z> = Z2x.quotient_ring(x^8+1) # Ring for rotations
print('Matrix for multiplication:\n',table([[f"{[3,1,1,2][(i+3-j)&3]:#0{4}x}" for i in range(4)] for j in range(4)]), sep='')

MCRR.<w> = PolynomialRing(RF); MCRR
MCR.<w> = MCRR.quotient_ring(w^4+1); MCR
a = (y+1)*w^3+w^2+w+y
print('a: ', end=''); pretty_print(a)
print('1/a: ', end=''); pretty_print(1/a)

def MixColumns(d, a):
    r = [[0 for j in range(4)] for i in range(4)]; r
    for i in range(4):
        cp = sum([d[i][j]*w^j for j in range(4)]) # create a polynomial in MCR
        cp = cp * a # mulitply with a
        
        cpl = cp.list() # write back to the column
        for j in range(4):
            r[i][j] = cpl[j]
    return r

ri = [[RF.random_element() for i in range(4)] for j in range(4)];
mixed = MixColumns(ri, a);
remixed = MixColumns(mixed, 1/a)
print('Input:', end=''); pretty_print(Matrix(ri))
print('After mixing columns:', end=''); pretty_print(Matrix(mixed))
print('After mixing columns using inverse:', end=''); pretty_print(Matrix(remixed))